# Data-Minding Group Progress Report

In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re

# Data Cleaning

## Summary of our data:

1. AQI Datasets

- aqi_county_2023: 325,393 rows × 10 columns — AQI by county
- aqi_county_2024: 206,919 rows × 10 columns — AQI by county.


2. Pollutant & Weather Datasets
Each of the following datasets has 29 columns and includes detailed info like site location, pollutant values, AQI, and metadata:

Pollutants: co, so2, ozone, no2

Weather: pressure, humidity_dp, wind, temperature

In [ ]:
# Define file paths
files_2024 = {
    "CO": "daily_42101_2024.csv",
    "SO2": "daily_42401_2024.csv",
    "Ozone": "daily_44201_2024.csv",
    "NO2": "daily_42602_2024.csv",
    "Pressure": "daily_PRESS_2024.csv",
    "Humidity_DewPoint": "daily_RH_DP_2024.csv",
    "Temperature": "daily_TEMP_2024.csv",
    "Wind": "daily_WIND_2024.csv"
}

# Load each dataset into individual DataFrames
CO_2024 = pd.read_csv(files_2024["CO"])
SO2_2024 = pd.read_csv(files_2024["SO2"])
Ozone_2024 = pd.read_csv(files_2024["Ozone"])
NO2_2024 = pd.read_csv(files_2024["NO2"])
Pressure_2024 = pd.read_csv(files_2024["Pressure"])
Humidity_DewPoint_2024 = pd.read_csv(files_2024["Humidity_DewPoint"])
Temperature_2024 = pd.read_csv(files_2024["Temperature"])
Wind_2024 = pd.read_csv(files_2024["Wind"])

# Example of checking the first few rows of each DataFrame
print(CO_2024.head())
print(SO2_2024.head())
print(Ozone_2024.head())
print(NO2_2024.head())
print(Pressure_2024.head())
print(Humidity_DewPoint_2024.head())
print(Temperature_2024.head())
print(Wind_2024.head())



<ipython-input-19-e256b74fd929>:14: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  CO_2024 = pd.read_csv(files["CO"])
<ipython-input-19-e256b74fd929>:15: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  SO2_2024 = pd.read_csv(files["SO2"])
<ipython-input-19-e256b74fd929>:16: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  Ozone_2024 = pd.read_csv(files["Ozone"])


   State Code  County Code  Site Num  Parameter Code  POC   Latitude  \
0           2           20        18           42101    1  61.205861   
1           2           20        18           42101    1  61.205861   
2           2           20        18           42101    1  61.205861   
3           2           20        18           42101    1  61.205861   
4           2           20        18           42101    1  61.205861   

    Longitude  Datum   Parameter Name Sample Duration  ... AQI Method Code  \
0 -149.824602  WGS84  Carbon monoxide          1 HOUR  ... NaN         554   
1 -149.824602  WGS84  Carbon monoxide          1 HOUR  ... NaN         554   
2 -149.824602  WGS84  Carbon monoxide          1 HOUR  ... NaN         554   
3 -149.824602  WGS84  Carbon monoxide          1 HOUR  ... NaN         554   
4 -149.824602  WGS84  Carbon monoxide          1 HOUR  ... NaN         554   

                                         Method Name  \
0  INSTRUMENTAL - Gas Filter Correlation T

In [ ]:
# Define file paths
files_2023 = {
    "CO": "daily_42101_2023.csv",
    "SO2": "daily_42401_2023.csv",
    "Ozone": "daily_44201_2023.csv",
    "NO2": "daily_42602_2023.csv",
    "Pressure": "daily_PRESS_2023.csv",
    "Humidity_DewPoint": "daily_RH_DP_2023.csv",
    "Temperature": "daily_TEMP_2023.csv",
    "Wind": "daily_WIND_2023.csv"
}

# Load each dataset into individual DataFrames
CO_2023 = pd.read_csv(files_2023["CO"])
SO2_2023 = pd.read_csv(files_2023["SO2"])
Ozone_2023 = pd.read_csv(files_2023["Ozone"])
NO2_2023 = pd.read_csv(files_2023["NO2"])
Pressure_2023 = pd.read_csv(files_2023["Pressure"])
Humidity_DewPoint_2023 = pd.read_csv(files_2023["Humidity_DewPoint"])
Temperature_2023 = pd.read_csv(files_2023["Temperature"])
Wind_2023 = pd.read_csv(files_2023["Wind"])

# Example of checking the first few rows of each DataFrame
print(CO_2023.head())
print(SO2_2023.head())
print(Ozone_2023.head())
print(NO2_2023.head())
print(Pressure_2023.head())
print(Humidity_DewPoint_2023.head())
print(Temperature_2023.head())
print(Wind_2023.head())



<ipython-input-30-3cd763aa6472>:14: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  CO_2023 = pd.read_csv(files_2023["CO"])
<ipython-input-30-3cd763aa6472>:15: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  SO2_2023 = pd.read_csv(files_2023["SO2"])
<ipython-input-30-3cd763aa6472>:16: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  Ozone_2023 = pd.read_csv(files_2023["Ozone"])
<ipython-input-30-3cd763aa6472>:17: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  NO2_2023 = pd.read_csv(files_2023["NO2"])


   State Code  County Code  Site Num  Parameter Code  POC   Latitude  \
0           1           73        23           42101    2  33.553056   
1           1           73        23           42101    2  33.553056   
2           1           73        23           42101    2  33.553056   
3           1           73        23           42101    2  33.553056   
4           1           73        23           42101    2  33.553056   

   Longitude  Datum   Parameter Name Sample Duration  ... AQI Method Code  \
0    -86.815  WGS84  Carbon monoxide          1 HOUR  ... NaN          93   
1    -86.815  WGS84  Carbon monoxide          1 HOUR  ... NaN          93   
2    -86.815  WGS84  Carbon monoxide          1 HOUR  ... NaN          93   
3    -86.815  WGS84  Carbon monoxide          1 HOUR  ... NaN          93   
4    -86.815  WGS84  Carbon monoxide          1 HOUR  ... NaN          93   

                                         Method Name   Local Site Name  \
0  INSTRUMENTAL - GAS FILTER C

## Common Characteristics
### Feature	Value
* Granularity: Daily-level per location (CBSA/County/Site)
* Temporality:	All datasets cover January–December 2023-2024. Some sites may have missing days or gaps, handled via imputation or aggregation.
* Scope:
  * Datasets span all of 2023 and 2024.
  * Covers multiple pollutants (CO, SO2, Ozone, NO2) and weather variables (Temperature, Wind, etc.).
  * Geographic scope is national (USA)
* Faithfulness:	Most datasets are complete (minimal to no missing values)


In [38]:
def clean_pollutant_dataset(df, pollutant_standard=None):
    # Optional filter for pollutant standard (e.g., CO 8-hour 1971)
    if pollutant_standard and 'Pollutant Standard' in df.columns:
        df = df[df['Pollutant Standard'] == pollutant_standard]

    # Convert 'Date Local' to datetime and drop rows with invalid dates
    if 'Date Local' in df.columns:
        df['Date Local'] = pd.to_datetime(df['Date Local'], errors='coerce')
        df = df.dropna(subset=['Date Local'])

    # Drop duplicate rows
    df = df.drop_duplicates()

    return df

def combine_datasets_POLLUTANTS(file1, file2, file3, file4, output_file):
    # Load and clean each dataset
    df1 = clean_pollutant_dataset(pd.read_csv(file1))  # Ozone
    df2 = clean_pollutant_dataset(pd.read_csv(file2), pollutant_standard='SO2 1-hour 2010')
    df3 = clean_pollutant_dataset(pd.read_csv(file3), pollutant_standard='CO 8-hour 1971')
    df4 = clean_pollutant_dataset(pd.read_csv(file4))  # NO2

    # Select relevant columns and rename for clarity
    df1 = df1[['State Name', 'County Name', 'Date Local', 'Local Site Name',
               'Arithmetic Mean', '1st Max Value', '1st Max Hour']].rename(
        columns={
            'Arithmetic Mean': 'Arithmetic Mean_ozone',
            '1st Max Value': '1st Max Value_ozone',
            '1st Max Hour': '1st Max Hour_ozone'
        })

    df2 = df2[['State Name', 'County Name', 'Date Local', 'Local Site Name',
               'Arithmetic Mean', '1st Max Value', '1st Max Hour']].rename(
        columns={
            'Arithmetic Mean': 'Arithmetic Mean_so2',
            '1st Max Value': '1st Max Value_so2',
            '1st Max Hour': '1st Max Hour_so2'
        })

    df3 = df3[['State Name', 'County Name', 'Date Local', 'Local Site Name',
               'Arithmetic Mean', '1st Max Value', '1st Max Hour']].rename(
        columns={
            'Arithmetic Mean': 'Arithmetic Mean_co',
            '1st Max Value': '1st Max Value_co',
            '1st Max Hour': '1st Max Hour_co'
        })

    df4 = df4[['State Name', 'County Name', 'Date Local', 'Local Site Name',
               'Arithmetic Mean', '1st Max Value', '1st Max Hour']].rename(
        columns={
            'Arithmetic Mean': 'Arithmetic Mean_no2',
            '1st Max Value': '1st Max Value_no2',
            '1st Max Hour': '1st Max Hour_no2'
        })

    # Merge datasets on common keys
    merged_df = pd.merge(df1, df2, on=['State Name', 'County Name', 'Date Local', 'Local Site Name'], how='inner')
    merged_df = pd.merge(merged_df, df3, on=['State Name', 'County Name', 'Date Local', 'Local Site Name'], how='inner')
    merged_df = pd.merge(merged_df, df4, on=['State Name', 'County Name', 'Date Local', 'Local Site Name'], how='inner')

    # Sort by location and date
    merged_df = merged_df.sort_values(by=['State Name', 'County Name', 'Date Local'])

    # Save to CSV
    merged_df.to_csv(output_file, index=False)
    print(f"Merged pollutant dataset saved to: {output_file}")


In [39]:
combine_datasets_POLLUTANTS(
    "daily_44201_2023.csv",  # Ozone
    "daily_42401_2023.csv",  # SO2
    "daily_42101_2023.csv",  # CO
    "daily_42602_2023.csv",  # NO2
    "merged_ALL_POLLUTANTS_2023.csv"
)

<ipython-input-38-4a3ea6a424ca>:18: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = clean_pollutant_dataset(pd.read_csv(file1))  # Ozone
<ipython-input-38-4a3ea6a424ca>:19: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = clean_pollutant_dataset(pd.read_csv(file2), pollutant_standard='SO2 1-hour 2010')
<ipython-input-38-4a3ea6a424ca>:20: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = clean_pollutant_dataset(pd.read_csv(file3), pollutant_standard='CO 8-hour 1971')
<ipython-input-38-4a3ea6a424ca>:21: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df4 = clean_pollutant_dataset(pd.read_csv(file4))  # NO2


Merged pollutant dataset saved to: merged_ALL_POLLUTANTS_2023.csv


In [40]:
combine_datasets_POLLUTANTS(
    "daily_44201_2024.csv",  # Ozone
    "daily_42401_2024.csv",  # SO2
    "daily_42101_2024.csv",  # CO
    "daily_42602_2024.csv",  # NO2
    "merged_ALL_POLLUTANTS_2024.csv"
)

<ipython-input-38-4a3ea6a424ca>:18: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = clean_pollutant_dataset(pd.read_csv(file1))  # Ozone
<ipython-input-38-4a3ea6a424ca>:19: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = clean_pollutant_dataset(pd.read_csv(file2), pollutant_standard='SO2 1-hour 2010')
<ipython-input-38-4a3ea6a424ca>:20: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = clean_pollutant_dataset(pd.read_csv(file3), pollutant_standard='CO 8-hour 1971')


Merged pollutant dataset saved to: merged_ALL_POLLUTANTS_2024.csv


In [41]:
def combine_datasets_METEO(file1, file2, file3, file4, output_file):
    # Read all four datasets
    df1 = pd.read_csv(file1)  #
    df2 = pd.read_csv(file2)  # SO2
    df3 = pd.read_csv(file3)  # CO
    df4 = pd.read_csv(file4)  # NO2
    df5 = df4.copy()

    # filtered_df = df[df['Ozone'] > 0.03]
    df4 = df4[df4['Parameter Name'] == 'Wind Speed - Resultant']
    df5 = df5[df5['Parameter Name'] == 'Wind Direction - Resultant']

    # Select the relevant columns and rename them for each dataset
    df1 = df1[['State Name', 'County Name', 'Date Local', 'Local Site Name', 'Arithmetic Mean', '1st Max Value', '1st Max Hour']].rename(
        columns={'Arithmetic Mean': 'Arithmetic Mean_PRESS', '1st Max Value': '1st Max Value_PRESS', '1st Max Hour': '1st Max Hour_PRESS'})
    df2 = df2[['State Name', 'County Name', 'Date Local', 'Local Site Name', 'Arithmetic Mean', '1st Max Value', '1st Max Hour']].rename(
        columns={'Arithmetic Mean': 'Arithmetic Mean_RH_DP', '1st Max Value': '1st Max Value_RH_DP', '1st Max Hour': '1st Max Hour_RH_DP'})
    df3 = df3[['State Name', 'County Name', 'Date Local', 'Local Site Name', 'Arithmetic Mean', '1st Max Value', '1st Max Hour']].rename(
        columns={'Arithmetic Mean': 'Arithmetic Mean_TEMP', '1st Max Value': '1st Max Value_TEMP', '1st Max Hour': '1st Max Hour_TEMP'})
    df4 = df4[['State Name', 'County Name', 'Date Local', 'Local Site Name', 'Arithmetic Mean', '1st Max Value', '1st Max Hour']].rename(
        columns={'Arithmetic Mean': 'Arithmetic Mean_WIND_SPEED', '1st Max Value': '1st Max Value_WIND_SPEED', '1st Max Hour': '1st Max Hour_WIND_SPEED'})
    df5 = df5[['State Name', 'County Name', 'Date Local', 'Local Site Name', 'Arithmetic Mean', '1st Max Value', '1st Max Hour']].rename(
        columns={'Arithmetic Mean': 'Arithmetic Mean_WIND_DIRECTION', '1st Max Value': '1st Max Value_WIND_DIRECTION', '1st Max Hour': '1st Max Hour_WIND_DIRECTION'})

    # Merge the datasets on common columns ('State Code', 'County Code', 'Date Local', 'Local Site Name')
    merged_df = pd.merge(df1, df2, on=['State Name', 'County Name', 'Date Local', 'Local Site Name'], how='inner')
    merged_df = pd.merge(merged_df, df3, on=['State Name', 'County Name', 'Date Local', 'Local Site Name'], how='inner')
    merged_df = pd.merge(merged_df, df4, on=['State Name', 'County Name', 'Date Local', 'Local Site Name'], how='inner')
    merged_df = pd.merge(merged_df, df5, on=['State Name', 'County Name', 'Date Local', 'Local Site Name'], how='inner')

    merged_df = merged_df.sort_values(by=['State Name', 'County Name', 'Date Local'], ascending=[True, True, True])

    # Save the combined dataset to a CSV file
    merged_df.to_csv(output_file, index=False)
    print(f"Merged dataset saved to {output_file}")

In [34]:
combine_datasets_METEO("daily_PRESS_2023.csv", "daily_RH_DP_2023.csv", "daily_TEMP_2023.csv", "daily_WIND_2023.csv", "merged_ALL_METEO_2023.csv")

Merged dataset saved to merged_ALL_METEO_2023.csv


In [35]:
combine_datasets_METEO("daily_PRESS_2024.csv", "daily_RH_DP_2024.csv", "daily_TEMP_2024.csv", "daily_WIND_2024.csv", "merged_ALL_METEO_2024.csv")

Merged dataset saved to merged_ALL_METEO_2024.csv


## Data Cleaning Steps

- Filtered pollutant standards (e.g., **CO 8-hour 1971**, **SO2 1-hour 2010**) and wind parameters (**Wind Speed - Resultant**, **Wind Direction - Resultant**) for consistency.  
- Selected key features: **Arithmetic Mean**, **Max Value**, and **Max Hour** across all datasets.  
- Renamed columns to reflect their source and ensure uniqueness after merging.  
- Dropped unnecessary columns like **Local Site Name** in final merges.  
- Handled missing values implicitly by performing **inner joins**—retaining only rows with complete data across all selected variables.


In [44]:
def combine_datasets(file1, file2, file3, output_file):
    # Read all three datasets: pollutants, meteorology, aqi_by_county
    df1 = pd.read_csv(file1)  # pollutants
    df2 = pd.read_csv(file2)  # meteorology
    df3 = pd.read_csv(file3)  # aqi

    df1 = df1.drop('Local Site Name', axis=1)
    df2 = df2.drop('Local Site Name', axis=1)
    df3 = df3[['State Name', 'county Name', 'Date', 'AQI', 'Category']].rename(columns={'county Name': 'County Name'})

    # Merge the datasets on common columns ('State Code', 'County Code', 'Date Local', 'Local Site Name')
    merged_df = pd.merge(df1, df2, on=['State Name', 'County Name', 'Date Local'], how='inner')
    merged_df = merged_df.rename(columns={'Date Local': 'Date'})
    merged_df = pd.merge(merged_df, df3, on=['State Name', 'County Name', 'Date'], how='inner')
    # merged_df = pd.merge(merged_df, df4, on=['State Name', 'County Name', 'Date Local', 'Local Site Name'], how='inner')

    merged_df = merged_df.sort_values(by=['State Name', 'County Name', 'Date'], ascending=[True, True, True])

    # Save the combined dataset to a CSV file
    merged_df.to_csv(output_file, index=False)
    print(f"Merged dataset saved to {output_file}")

In [47]:
combine_datasets("merged_ALL_METEO_2023.csv", "merged_ALL_POLLUTANTS_2023.csv", "daily_aqi_by_county_2023.csv", "all_attributes_2023.csv")

Merged dataset saved to all_attributes_2023.csv


In [45]:
combine_datasets("merged_ALL_METEO_2024.csv", "merged_ALL_POLLUTANTS_2024.csv", "daily_aqi_by_county_2024.csv", "all_attributes_2024.csv")

Merged dataset saved to all_attributes_2024.csv


In [48]:
df = pd.read_csv('all_attributes_2023.csv')

unique_states = df['State Name'].unique()

df1 = pd.read_csv('all_attributes_2024.csv')

unique_states1 = df1['State Name'].unique()

print(f"Number of unique states: {unique_states}")

print(f"Number of unique states: {unique_states1}")

count = 0
commonState = []
for item in unique_states1:
    for i in unique_states:
        if (item == i):
            count += 1
            commonState.append(item)
            break

print(count)
print(len(commonState))
print(commonState)

Number of unique states: ['Alabama' 'Arizona' 'California' 'Connecticut' 'District Of Columbia'
 'Georgia' 'Idaho' 'Illinois' 'Indiana' 'Kentucky' 'Louisiana' 'Maryland'
 'Massachusetts' 'Michigan' 'Missouri' 'Nevada' 'New Hampshire'
 'New Mexico' 'North Carolina' 'North Dakota' 'Ohio' 'Pennsylvania'
 'Rhode Island' 'Tennessee' 'Texas' 'Virginia' 'Washington' 'Wyoming']
Number of unique states: ['Arizona' 'California' 'Connecticut' 'Georgia' 'Idaho' 'Illinois'
 'Indiana' 'Iowa' 'Louisiana' 'Maryland' 'Massachusetts' 'Michigan'
 'Missouri' 'Nevada' 'New Hampshire' 'New Mexico' 'North Carolina'
 'North Dakota' 'Ohio' 'Oregon' 'Pennsylvania' 'Rhode Island' 'Texas'
 'Virginia' 'Washington' 'Wyoming']
24
24
['Arizona', 'California', 'Connecticut', 'Georgia', 'Idaho', 'Illinois', 'Indiana', 'Louisiana', 'Maryland', 'Massachusetts', 'Michigan', 'Missouri', 'Nevada', 'New Hampshire', 'New Mexico', 'North Carolina', 'North Dakota', 'Ohio', 'Pennsylvania', 'Rhode Island', 'Texas', 'Virginia', 'W

In [50]:
def filterRows(filename):
    commonState = ['Arizona', 'California', 'Connecticut', 'Georgia', 'Idaho', 'Illinois', 'Indiana', 'Louisiana', 'Maryland', 'Massachusetts', 'Michigan', 'Missouri', 'Nevada', 'New Hampshire', 'New Mexico', 'North Carolina', 'North Dakota', 'Ohio', 'Pennsylvania', 'Rhode Island', 'Texas', 'Virginia', 'Washington', 'Wyoming']

    match = re.search(r'_(\d{4})', filename)
    year = match.group(1)

    df = pd.read_csv(filename)

    # Filter rows where 'State Name' is in the commonState list
    filtered_df = df[df['State Name'].isin(commonState)]

    # Optional: Save to a new CSV file
    output_filename = f'24StateAQI_{year}.csv'
    filtered_df.to_csv(output_filename, index=False)

    # Display the first few rows to verify
    print(filtered_df.head())

In [54]:
filterRows("all_attributes_2023.csv")
filterRows("all_attributes_2024.csv")

    State Name County Name        Date  Arithmetic Mean_PRESS  \
533    Arizona    Maricopa  2023-01-01             967.183333   
534    Arizona    Maricopa  2023-01-01             967.183333   
535    Arizona    Maricopa  2023-01-02             973.091667   
536    Arizona    Maricopa  2023-01-02             973.091667   
537    Arizona    Maricopa  2023-01-03             978.025000   

     1st Max Value_PRESS  1st Max Hour_PRESS  Arithmetic Mean_RH_DP  \
533                970.3                   0              82.083333   
534                970.3                   0              82.083333   
535                976.5                  21              71.916667   
536                976.5                  21              71.916667   
537                981.0                  22              68.458333   

     1st Max Value_RH_DP  1st Max Hour_RH_DP  Arithmetic Mean_TEMP  ...  \
533                 90.0                  16             54.041667  ...   
534                 90.0        